In [1]:
# api_key = "https://developer.nrel.gov/api/alt-fuel-stations/v1.json?fuel_type=E85,ELEC&state=CA&limit=2&api_key=VndyeRDWmKaUeuwDuwhPR3IRIpBpoF7tMs4r2Zc7&format=JSON"
# https://developer.nrel.gov/docs/transportation/alt-fuel-stations-v1/all/#response-fields

In [2]:
import requests
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import math

# Section 1: Data Preparation #
### Data option 1 : Reading data from saved dataFrame  ###

In [ ]:
# EV Charging Station Data
df_nrel = pd.read_excel("nrel_gov_data.xlsx")
# Electricity Cost Data
df_eia = pd.read_excel("eia_gov.xlsx")

### Data option 2 : Download data from web ###

#### EV Charging Station Dataset ####

In [ ]:
URL = "https://developer.nrel.gov/api/alt-fuel-stations/v1.json?api_key=VndyeRDWmKaUeuwDuwhPR3IRIpBpoF7tMs4r2Zc7"

In [ ]:
response = requests.get(url = URL)

In [ ]:
# Saving data to files as backup
# data_file = open("data.json",'w')
# data_file.write(str(response.json()))
# data_file.close()

In [ ]:
data_dict = response.json()

In [ ]:
print(data_dict.keys())

In [ ]:
print(data_dict['station_locator_url'])

In [ ]:
print(data_dict['total_results'])

In [ ]:
print(data_dict['station_counts'])

In [ ]:
df = pd.DataFrame(data_dict["fuel_stations"])

In [ ]:
df.to_excel("nrel_gov_data.xlsx")

#### Electricity Cost Dataset ####

In [ ]:
# EIA
state_list = ["AK","AL","AR","AZ","CA","CO","CT","DC","DE","FL","GA","HI","IA","ID", "IL","IN","KS","KY","LA","MA","MD","ME","MI","MN","MO","MS","MT","NC","ND","NE","NH","NJ","NM","NV","NY", "OH","OK","OR","PA","RI","SC","SD","TN","TX","UT","VA","VT","WA","WI","WV","WY"]

In [ ]:
data_dict = dict()
for state in state_list:
    URL = "http://api.eia.gov/series/?api_key={}&series_id=ELEC.PRICE.{}-ALL.M".format(eia_api_key, state)
    response = requests.get(url = URL)
    print((state, response))
    data = response.json()['series'][0]['data']
    for entry in data:
        if entry[0] not in data_dict:
            data_dict[entry[0]] = dict()
        data_dict[entry[0]][state] = entry[1]


In [ ]:
df_eia = pd.DataFrame(data_dict)
df_eia.to_excel("eia_gov.xlsx")

In [ ]:
df_eia

# Section 2: Data clean-up and visualization #

In [ ]:
print(df_nrel)

In [ ]:
df_nrel.size

In [ ]:
df_nrel.shape

In [ ]:
df_nrel.columns

### Fields of interest ###
- fuel_type_code: choose "ELEC" for EV stations
- city
- state
- zip
- country: US or CA
- status_code: E = Open, P = Planned, T = Temporarily unavailable
- access_code: public, private
- owner_type_code:
P = Privately owned,
T = Utility owned,
FG = Federal government owned,
LG = Local/Municipal government owned,
SG = State/Provincial government owned,
J = Jointly owned (combination of owner types)
- ev_level1_evse_num: the number of Level 1 EVSE (standard 110V outlet)
- ev_level2_evse_num: the number of Level 2 EVSE (J1772 connector)
- ev_dc_fast_num: the number of DC Fast Chargers
- ev_connector_types
- ev_network: the name of the EVSE network, if applicable
- ev_pricing: information about whether and how much users must pay to use the EVSE.
- ev_renewable_source:  the type of renewable energy used to generate electricity on-site, given as code values as described below:
GEOTHERMAL = Geothermal,
HYDRO = Hydropower,
LANDFILL = Landfill,
LIVESTOCK = Livestock Operations,
SOLAR = Solar,
WASTEWATER = Wastewater Treatment,
WIND = Wind
- geocode_status: A rating indicating the approximate accuracy of the latitude and longitude for the station's address, given as code values as described below:
- latitude
- longitude
- open_date
- facility_type

In [ ]:
field_of_interest = ['fuel_type_code', 'city', 'state', 
                     'zip', 'country', 'status_code', 
                     'access_code', 'owner_type_code', 
                     'ev_level1_evse_num', 'ev_level2_evse_num', 'ev_dc_fast_num', 
                     'ev_connector_types', 'ev_network', 'ev_pricing', 
                     'ev_renewable_source', 'geocode_status', 'latitude', 
                     'longitude', 'open_date', 'facility_type']

In [ ]:
EV_stations = df_nrel[df_nrel['fuel_type_code'] == 'ELEC']
EV_stations

In [ ]:
print(EV_stations.shape)
# We have 26667 EV station records

In [ ]:
EV_stations = EV_stations[field_of_interest]
EV_stations

In [ ]:
EV_stations[EV_stations['open_date'].notnull()].shape[0]

In [ ]:
# how many ev stations have pricing info
EV_stations[EV_stations['ev_pricing'].notnull()].shape[0]

In [ ]:
facility_dist_dict = dict()
for facility in EV_stations['state']:
    if facility not in facility_dist_dict:
        facility_dist_dict[facility] = 1
    facility_dist_dict[facility] += 1
df_dict = dict()
df_dict['state'] = list(facility_dist_dict.keys())
df_dict['count'] = list(facility_dist_dict.values())
facility_dist_df = pd.DataFrame(df_dict)
facility_dist_df

fig = go.Figure(data=go.Choropleth(
    locations=facility_dist_df['state'], # Spatial coordinates
    z = facility_dist_df['count'].astype(float), # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = 'Reds',
    colorbar_title = "",
))

fig.update_layout(
    title_text = 'EV Charging Facility',
    geo_scope='usa', # limite map scope to USA
)

fig.show()

In [ ]:
EV_stations[['state', 'ev_level1_evse_num']]

In [ ]:
# ev_level1_evse_num	ev_level2_evse_num	ev_dc_fast_num
facility_dist_dict = dict()
for facility in EV_stations['state']:
    if facility not in facility_dist_dict:
        facility_dist_dict[facility] = 0
    facility_dist_dict[facility] += 1
df_dict = dict()
df_dict['state'] = list(facility_dist_dict.keys())
df_dict['count'] = list(facility_dist_dict.values())
facility_dist_df = pd.DataFrame(df_dict)
facility_dist_df

fig = go.Figure(data=go.Choropleth(
    locations=facility_dist_df['state'], # Spatial coordinates
    z = facility_dist_df['count'].astype(float), # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = 'Reds',
    colorbar_title = "",
))

fig.update_layout(
    title_text = 'EV Charging Facility',
    geo_scope='usa', # limite map scope to USA
)

fig.show()

In [ ]:
station_dist_dict = dict()
for station in EV_stations[['state','ev_level1_evse_num','ev_level2_evse_num','ev_dc_fast_num']].iterrows():
    if station[1]['state'] not in station_dist_dict:
        station_dist_dict[station[1]['state']] = 0
    if not math.isnan(station[1]['ev_level1_evse_num']):
        station_dist_dict[station[1]['state']] += station[1]['ev_level1_evse_num']
    if not math.isnan(station[1]['ev_level2_evse_num']):
        station_dist_dict[station[1]['state']] += station[1]['ev_level2_evse_num']
    if not math.isnan(station[1]['ev_dc_fast_num']):
        facility_dist_dict[station[1]['state']] += station[1]['ev_dc_fast_num']
# print(station_dist_dict)
df_dict = dict()
df_dict['state'] = list(station_dist_dict.keys())
df_dict['count'] = list(station_dist_dict.values())
station_dist_df = pd.DataFrame(df_dict)
station_dist_df

fig = go.Figure(data=go.Choropleth(
    locations=station_dist_df['state'], # Spatial coordinates
    z = station_dist_df['count'].astype(float), # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = 'Reds',
    colorbar_title = "Number of Charging Facilities",
))

fig.update_layout(
    title_text = 'EV Charging Stations',
    geo_scope='usa', # limite map scope to USA
)

fig.show()

In [ ]:
avg_size_dist_dict = dict()
for item in station_dist_dict.items():
    avg_size_dist_dict[item[0]] = item[1] / facility_dist_dict[item[0]]

df_dict['state'] = list(avg_size_dist_dict.keys())
df_dict['count'] = list(avg_size_dist_dict.values())
avg_size_dist_df = pd.DataFrame(df_dict)
avg_size_dist_df

fig = go.Figure(data=go.Choropleth(
    locations=avg_size_dist_df['state'], # Spatial coordinates
    z = avg_size_dist_df['count'].astype(float), # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = 'Reds',
    colorbar_title = "Stations per Site",
))

fig.update_layout(
    title_text = 'Average Charging Facility Size',
    geo_scope='usa', # limite map scope to USA
)

fig.show()

#### EIA Electricity Cost Dataset ####

In [ ]:
fig = go.Figure(data=go.Choropleth(
    locations=df_eia.index, # Spatial coordinates
    z = df_eia['201908'].astype(float), # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = 'Reds',
    colorbar_title = "USD per kWh",
))

fig.update_layout(
    title_text = 'August 2019 US Electricity Cost',
    geo_scope='usa', # limite map scope to USA
)

fig.show()

In [ ]:
# Create figure
fig = go.Figure()

# Add traces, one for each slider step
for step in list(df_eia.columns):
    
    fig.add_trace(
        go.Choropleth(
            locations=df_eia.index, # Spatial coordinates
            z = df_eia[step].astype(float), # Data to be color-coded
            locationmode = 'USA-states', # set of locations match entries in `locations`
            colorscale = 'Reds',
            colorbar_title = "USD/kWh",
            name = step))

# Make 10th trace visible
fig.data[0].visible = True

# Create and add slider
steps = []
for i in range(len(fig.data)):
    step = dict(
        method="restyle",
        args=["visible", [False] * len(fig.data)],
    )
    step["args"][1][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=0,
#     currentvalue={"201908"},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(
    sliders=sliders,
    title_text = '2011-2019 Monthly Electricity Cost in the US',
    geo_scope='usa' # limite map scope to USA
)

fig.show()